# IMDB
https://zhuanlan.zhihu.com/p/63852350?utm_source=wechat_session&utm_medium=social&utm_oi=746820673451597824

In [3]:
import os
datapath = r'aclImdb\train'
pos_files = os.listdir(datapath+'/pos')
neg_files = os.listdir(datapath+'/neg')
print(len(pos_files)) # 训练集中正面数据集数
print(len(neg_files))

12500
12500


In [4]:
import numpy as np
pos_all = []
neg_all = []
for pf,nf in zip(pos_files,neg_files):
    with open(datapath+'/pos'+'/'+pf,encoding='utf-8') as f:
        s = f.read()
        # 合并正面数据
        pos_all.append(s)
    with open(datapath+'/neg'+'/'+nf,encoding='utf-8') as f:
        s = f.read()
        # 合并负面数据
        neg_all.append(s)
print(len(pos_all))
print(len(neg_all))
X_orig = np.array(pos_all+neg_all)
Y_orig = np.array([1 for _ in range(12500)] + [0 for _ in range(12500)]) #  用于判断？？
print("X_orig:",X_orig.shape)
print("Y_orig:",Y_orig.shape)

12500
12500
X_orig: (25000,)
Y_orig: (25000,)


In [7]:
# 预处理
from keras.preprocessing.text import text_to_word_sequence,one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
import time
vocab_size = 20000
maxlen = 200
print("Start fitting the corpus......")
t = Tokenizer(vocab_size) # 要使得文本向量化时省略掉低频词，就要设置这个参数
tik = time.time()
t.fit_on_texts(X_orig) # 在所有的评论数据集上训练，得到统计信息
tok = time.time()
word_index = t.word_index # 不受vocab_size的影响
print('all_vocab_size',len(word_index))
print("Fitting time: ",(tok-tik),'s')
print("Start vectorizing the sentences.......")
v_X = t.texts_to_sequences(X_orig) # 受vocab_size的影响
print("Start padding......")
pad_X = pad_sequences(v_X,maxlen=maxlen,padding='post')
print("Finished!")

Start fitting the corpus......
all_vocab_size 166
Fitting time:  6.042072772979736 s
Start vectorizing the sentences.......
Start padding......
Finished!


In [8]:
import copy
x = list(t.word_counts.items())
s = sorted(x,key=lambda p:p[1],reverse=True) # 递减
small_word_index = copy.deepcopy(word_index) # 防止原来的字典也被改变了
print("Removing less freq words from word-index dict...")
for item in s[20000:]:
    small_word_index.pop(item[0]) # 除去最低频的词？？
print("Finished!")
print(len(small_word_index))
print(len(word_index))

Removing less freq words from word-index dict...
Finished!
166
166


In [ ]:
import gensim
model_file = 'D:\\GoogleNews-vectors-negative300.bin'
print("Loading word2vec model......")
wv_model = gensim.models.KeyedVectors.load_word2vec_format(model_file,binary=True)

Loading word2vec model......


In [ ]:
embedding_matrix = np.random.uniform(size=(vocab_size+1,300)) # +1是要留一个给index=0
print("Transfering to the embedding matrix......")
# sorted_small_index = sorted(list(small_word_index.items()),key=lambda x:x[1])
for word,index in small_word_index.items():
    try:
        word_vector = wv_model[word]
        embedding_matrix[index] = word_vector
    except:
        print("Word: [",word,"] not in wvmodel! Use random embedding instead.")
print("Finished!")
print("Embedding matrix shape:\n",embedding_matrix.shape)

In [ ]:
from sklearn.model_selection import train_test_split
np.random.seed = 1
random_indexs = np.random.permutation(len(pad_X))
X = pad_X[random_indexs]
Y = Y_orig[random_indexs]
print(Y[:50])
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)
print("X_train:",X_train.shape)
print("y_train:",y_train.shape)
print("X_test:",X_test.shape)
print("y_test:",y_test.shape)
print(list(y_train).count(1))
print(list(y_train).count(0))

In [ ]:
import keras
from keras.models import Sequential,Model
from keras.layers import Input,Dense,GRU,LSTM,Activation,Dropout,Embedding
from keras.layers import Multiply,Concatenate,Dot

inputs = Input(shape=(maxlen,))
use_pretrained_wv = True
if use_pretrained_wv:
    wv = Embedding(VOCAB_SIZE+1,wv_dim,input_length=MAXLEN,weights=[embedding_matrix])      (inputs)
else:
    wv = Embedding(VOCAB_SIZE+1,wv_dim,input_length=MAXLEN)(inputs)

h = LSTM(128)(wv)
y = Dense(1,activation='sigmoid')(h)
m = Model(input=inputs,output=y)
m.summary()

m.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
m.fit(X_train,y_train,batch_size=32,epochs=3,validation_split=0.15)